# Chinook: Answering Business Questions Using SQL

In this guided project, we're going to practice using our SQL skills to answer business questions.
Chinook is a sample database available for SQL Server, Oracle, MySQL, etc.  
The data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

## Summary
- Albums from Meteor and the Girls, Red Tone, and Slim Jim Bites should yield the most sales from the newly signed record label.
- The differences in sales across each agents is roughly explained by their hiring dates.
- USA has generated the most sales over time.
- Countries in 'Other' category should be noted as they make up a significant amount after addition.
- Growth shown in countries with high cus_order_lifetime (high spend), and low total_customers.
- It is not advisable for management to switch their purchasing strategy to only acquire the most popular track, as 57% of invoices are album purchases

###  Lets start by reading in the dataset.

In [28]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [29]:
%%sql
SELECT
    name,
    type,
    sql
FROM sqlite_master
WHERE type IN ("table");

Done.


name,type,sql
album,table,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
artist,table,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
customer,table,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
employee,table,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
genre,table,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
invoice,table,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
invoice_line,table,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
media_type,table,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
playlist,table,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
playlist_track,table,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


## New Record Label

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| --- | --- |
|Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop|
| Slim Jim Bites | Blues |


The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [30]:
%%sql
/* Genres with the most tracks sold in the USA */

SELECT g.name AS genre_name, g.genre_id AS genre_id, SUM(i.quantity) AS number_of_sold_tracks
FROM genre g
INNER JOIN track t ON g.genre_id = t.genre_id
LEFT JOIN invoice_line i ON i.track_id = t.track_id
GROUP BY t.genre_id
ORDER BY 3 DESC

Done.


genre_name,genre_id,number_of_sold_tracks
Rock,1,2635
Metal,3,619
Alternative & Punk,4,492
Latin,7,167
R&B/Soul,14,159
Blues,6,124
Jazz,2,121
Alternative,23,117
Easy Listening,12,74
Pop,9,63


### Observations
We can see that Punk, Blues, and Pop are the top three with consideration of the artists we signed. Therefore, purchasing album's from Red Tone, Slim Jim Bites, Meteor and the Girls in that order should yield the most optimal results.

## Sales Support Agents (SSA)
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [31]:
%%sql
/* Show all Support Agents and Sales */

SELECT e.employee_id, 
       e.first_name || ' ' || e.last_name AS 'name', 
       e.title, e.reports_to, 
       e.hire_date, 
       ROUND(SUM(i.total), 2) AS 'Total_Sales'
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 1

Done.


employee_id,name,title,reports_to,hire_date,Total_Sales
3,Jane Peacock,Sales Support Agent,2,2017-04-01 00:00:00,1731.51
4,Margaret Park,Sales Support Agent,2,2017-05-03 00:00:00,1584.0
5,Steve Johnson,Sales Support Agent,2,2017-10-17 00:00:00,1393.92


### Observations

The task requested was to find the total dollar amount of sales assigned to each support agent and to determine performance. All SSA's started in the same year, with Jane P. having the most sales followed by Margaret, and Steve. The difference may correspond with their hiring dates.  More metrics can be brought in for analysis such as if any SSA's are showing growth in their sales over time.

Below is a table joining all customer's invoice dates to assigned employees. We can analyze below to see if SSA's show more sales from a customer after they've been assigned to determine other trends.

In [32]:
%%sql
/* Show all invoices respective to each customer and SSAs. Limited to 10 to simplify report. */
/* Raw data could be plotted on a visualization */

SELECT e.first_name || ' ' || e.last_name AS Employee,
       c.first_name || ' ' || c.Last_name AS Customer,
       i.invoice_date AS 'Invoice Date', 
       ROUND(i.total, 2) AS Total
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 1, 2, 3
LIMIT 10

Done.


Employee,Customer,Invoice Date,Total
Jane Peacock,Edward Francis,2017-01-03 00:00:00,9.9
Jane Peacock,Edward Francis,2017-02-21 00:00:00,12.87
Jane Peacock,Edward Francis,2017-06-20 00:00:00,7.92
Jane Peacock,Edward Francis,2017-08-27 00:00:00,4.95
Jane Peacock,Edward Francis,2018-12-14 00:00:00,5.94
Jane Peacock,Edward Francis,2019-01-24 00:00:00,4.95
Jane Peacock,Edward Francis,2019-07-20 00:00:00,3.96
Jane Peacock,Edward Francis,2020-01-19 00:00:00,3.96
Jane Peacock,Edward Francis,2020-06-02 00:00:00,9.9
Jane Peacock,Edward Francis,2020-08-02 00:00:00,10.89


## Sales by Country
Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value


In [33]:
%%sql
/* This query used to sort the group the sales data by invoice_id and filter countries with one customer to 'other' */
/* This query is also necessary to stage for the original requested task */

SELECT CASE WHEN (
                  SELECT COUNT(*)
                  FROM customer
                  WHERE country = c.country
                 ) = 1 THEN 'Other'
            ELSE country
       END AS country,
       c.customer_id AS customer_id,
       i.invoice_id AS invoice_id,
       i.total AS total
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
LIMIT 10

Done.


country,customer_id,invoice_id,total
USA,18,1,15.84
Canada,30,2,9.9
France,40,3,1.98
USA,18,4,7.92
USA,27,5,16.83
Canada,31,6,1.98
Other,49,7,10.89
India,59,8,9.9
USA,18,9,8.91
Canada,31,10,1.98


In [34]:
%%sql
/* Final Cell to calcualte sales by Country */


/* This subquery is the same as the mentioned above query that is necessary */

WITH country_other AS
(
SELECT CASE WHEN (
                  SELECT COUNT(*)
                  FROM customer
                  WHERE country = c.country
                 ) = 1 THEN 'Other'
            ELSE country
       END AS country,
       c.customer_id AS customer_id,
       i.invoice_id AS invoice_id,
       i.total AS total
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
)


/* Calculate Sales by Country, with countries with one customer labelled as other and put in 'other' group */

SELECT
    country,
    COUNT(DISTINCT customer_id) AS total_customers,
    ROUND(SUM(total), 2) AS total_sales,
    ROUND(SUM(total) / COUNT(distinct customer_id), 2) AS cus_order_lifetime,
    ROUND(SUM(total) / COUNT(DISTINCT invoice_id), 2) AS avg_inv_order
FROM (
      SELECT *,
             CASE WHEN country = 'Other' THEN 1 ELSE 0
             END AS sort
      FROM country_other
     )
GROUP BY country
ORDER by sort, total_sales DESC

Done.


country,total_customers,total_sales,cus_order_lifetime,avg_inv_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


### Observations:
USA has generated the most sales over time, along with having the most customers in one country. Growth is also shown in countries where cus_order_lifetime is high, while total_customers is low in places such as Czech, India, and Portugal. Countries categorized in 'other' also make up for a significant amount of sales, and should be noted.

## New Purchasing Strategy

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [35]:
%%sql
/* Subquery selects first track from each invoice */

WITH invoice_first_track AS
(
SELECT invoice_id, 
       MIN(track_id) AS track_id
FROM invoice_line
GROUP BY 1
)

/* Determines whether if invoice was an album purchase or not */

SELECT Album_purchase,
       COUNT(invoice_id) AS 'Number_of_invoices',
       ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice), 2) AS 'Percentage'
FROM
(
    SELECT ifs.*,
       CASE
        WHEN(
            SELECT t.track_id
            FROM track t
            WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                WHERE t2.album_id = ifs.track_id)
        
            EXCEPT
            
            SELECT track_id FROM invoice_line
            WHERE invoice_id = ifs.invoice_id
            ) IS NULL THEN 'Yes'
        ELSE 'No'
        END AS 'Album_purchase'
    FROM invoice_first_track AS ifs
)
GROUP BY 1

Done.


Album_purchase,Number_of_invoices,Percentage
No,264,0.43
Yes,350,0.57


### Observation
Album and non-Album purchases are split 57% and 43% respectively. Management's purchasing strategy of only acquiring the most popular tracks should not be advised, given that 57% of invoices are album purchases

## Summary
- Albums from Meteor and the Girls, Red Tone, and Slim Jim Bites should yield the most sales from the newly signed record label.
- The differences in sales across each agents is roughly explained by their hiring dates.
- USA has generated the most sales over time.
- Countries in 'Other' category should be noted as they make up a significant amount after addition.
- Growth shown in countries with high cus_order_lifetime (high spend), and low total_customers.
- It is not advisable for management to switch their purchasing strategy to only acquire the most popular track, as 57% of invoices are album purchases